In [1]:
# Import necessary libraries

import pandas as pd
import re

In [3]:
# Set the name of the file to read and read it into a DataFrame

file_name = 'red_wines'

df = pd.read_csv(file_name +'.csv', encoding='ansi', sep=';', header=None, names=['company', 'name', 'avg_review', 'num_review', 'breads', 'price', 'level_structure', 'food_advice', 'n_m'])

In [4]:
df.head()

,company,name,avg_review,num_review,breads,price,level_structure,food_advice,n_m
0,Podernuovo a Palazzone,Sotirio 2011,"4,3",75 Voti,Italia·Toscana·Podernuovo a Palazzone·Vino ros...,"32,90","[('Leggero-Strutturato', '56.456%'), ('Morbido...",Manzo / Agnello / Vitello,"[('legno', 20), ('Erbaceo', 14), ('bacche nere..."
1,Gaja,Pieve Santa Restituta Sugarille Brunello di Mo...,"4,3",27 Voti,Italia·Brunello di Montalcino·Gaja·Vino rosso·...,"179,00","[('Leggero-Strutturato', '52.7566%'), ('Morbid...","Manzo / Agnello / Cacciagione (cervo, camoscio)","[('legno', 185), ('Erbaceo', 183), ('frutti ro..."
2,Le Macchiole,Paleo 2017,"4,5",301 Voti,Italia·Toscana·Le Macchiole·Vino rosso·Caberne...,"109,00","[('Leggero-Strutturato', '58.253%'), ('Morbido...",Manzo / Agnello / Vitello,"[('legno', 650), ('bacche nere', 471), ('Erbac..."
3,Tenuta San Guido,Sassicaia 2000,"4,6",2174 Voti,Italia·Bolgheri Sassicaia·Tenuta San Guido·Vin...,"569,25","[('Leggero-Strutturato', '62.0472%'), ('Morbid...",Manzo / Pasta / Agnello,"[('legno', 3368), ('bacche nere', 3033), ('Erb..."
4,Elio Altare,Cerretta Barolo Riserva (Vigna Bricco) 2013,"4,4",153 Voti,Italia·Barolo·Elio Altare·Vino rosso·Nebbiolo,"161,00","[('Leggero-Strutturato', '63.0586%'), ('Morbid...",Manzo / Pasta / Agnello,"[('Erbaceo', 118), ('legno', 111), ('frutti ro..."


In [5]:
# Remove rows where avg_review is 'na'
df = df[df.avg_review!='na']

# Convert avg_review to float, replacing commas with dots
df[['avg_review']] = df[['avg_review']].replace(value='.', regex=',').astype(float)

# Convert num_review to int, removing the string 'Voti'
df['num_review'] = df['num_review'].apply(lambda x : int(x.split()[0]))



In [6]:
# Convert price to float, removing the euro symbol and replacing commas with dots
df = df[df.price!='na']
df[['price']] = df[['price']].replace(value='.', regex=',').replace(value='', regex='€').astype(float)

In [7]:
# Split the 'breads' column into separate columns
df['country'] = df['breads'].apply(lambda x : x.split('·')[0])
df['region'] = df['breads'].apply(lambda x : x.split('·')[1])
df['wine_type'] = df['breads'].apply(lambda x : x.split('·')[-1])


In [9]:
# Split 'name' into 'name' and 'year' for red wines

def clean_year(x):
    if x.split()[-1].isdigit():
        x = int(x.split()[-1])
    else:
        x = 'na'
    return x

def clean_name(x):
    if x.split()[-1].isdigit():
        x = x.replace(x.split()[-1], '')
    else:
        x = x
    return x

df['year'] = df['name'].apply(clean_year)
df['name'] = df['name'].apply(clean_name)

In [ ]:
# Split 'name' into 'name' and 'year' for white wines

def clean_year(x):
    if x.split()[-1] in ('N.V.', 'Blanc'):
        return None
    else:
        return str(x.split()[-1])

df['year'] = df['name'].apply(clean_year)
df['name'] = df['name'].apply(lambda x : ' '.join(x.split()[:-1]))

In [10]:
# Create a new column for each element of 'level_structure'

df['legg_strutt'] = df['level_structure'].apply(lambda x : None if x=='[]' else float(re.sub('[\'%),px]', '', x.split()[1])))
df['secco_dolce'] = df['level_structure'].apply(lambda x : None if x=='[]' else float(re.sub('[\'%),px]', '', x.split()[3])))
df['piatto_acid'] = df['level_structure'].apply(lambda x : None if x=='[]' else float(re.sub('[\'%),px\]]', '', x.split()[5])))


In [11]:
# Create a new column for the cleaned notes

def clean_notes(x):
    if x == '[]':
        return None
    else:
        notes = []
        for element in x.split('),'):
            notes.append(re.sub('[\[\'(,]', '', element.split()[0]))
        return '/'.join(notes)

df['notes'] = df['n_m'].apply(clean_notes)   


In [12]:
# Create a new column for the cleaned notes

def clean_notes(x):
    if x == '[]':
        return None
    else:
        notes = []
        for element in x.split('),'):
            
            notes.append(element.split('\'')[1])
        return '/'.join(notes)

df['notes'] = df['n_m'].apply(clean_notes)   


In [14]:
# Drop unnecessary columns

df = df.drop(columns=['breads', 'level_structure', 'n_m'], axis=1)

In [16]:
# Write the cleaned DataFrame to a new CSV file

df.to_csv(file_name+'_clean.csv', sep=';', encoding='ansi', index=False)